In [31]:
import numpy as np
import pandas as pd
from dataLoad import PulsarData
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score

Read in the pulsar data from the dataLoad.py script

In [32]:
features_data = PulsarData('HTRU_2').features
targets_data = PulsarData('HTRU_2').targets

Shuffle and split the data into training and test groups with 3:1 split

In [33]:
train_features_data, test_features_data, train_targets_data, test_targets_data  =  train_test_split( features_data, 
                                                        targets_data, test_size=0.25, random_state=42)

In [34]:
def GBC_CrossValidation(n_estimators, learning_rate, max_depth, data, targets):
    """Decision Tree cross validation.
       Fits a Decision Tree with the given paramaters to the target 
       given data, calculated a CV accuracy score and returns the mean.
       The goal is to find combinations of max_depth, min_samples_leaf 
       that maximize the accuracy
    """
    
    estimator = GradientBoostingClassifier(n_estimators=n_estimators, learning_rate=learning_rate, 
                                 max_depth=max_depth, random_state=0)
    
    cval = cross_val_score(estimator, data, targets, scoring='accuracy', cv=5)
    
    return cval.mean()

In [35]:
def optimize_GBC(data, targets, pars, n_iter=5):
    """Apply Bayesian Optimization to Decision Tree parameters."""
    
    def crossval_wrapper(n_estimators, learning_rate, max_depth):
        """Wrapper of Decision Tree cross validation. 
           Notice how we ensure max_depth, min_samples_leaf 
           are casted to integer before we pass them along.
        """
        return GBC_CrossValidation(n_estimators=int(n_estimators), 
                                            learning_rate=learning_rate, 
                                            max_depth=int(max_depth),
                                            data=data, 
                                            targets=targets)

    optimizer = BayesianOptimization(f=crossval_wrapper, 
                                     pbounds=pars, 
                                     random_state=42, 
                                     verbose=2)
    optimizer.maximize(init_points=4, n_iter=n_iter)

    return optimizer

In [36]:
parameters_BayesianOptimization = {"learning_rate": (0.0001, 1), 
                                   "max_depth": (1, 5),
                                   "n_estimators": (50,500)
                                  }

BayesianOptimization = optimize_GBC(train_features_data, 
                                             train_targets_data, 
                                             parameters_BayesianOptimization, 
                                             n_iter=5)
print(BayesianOptimization.max)

|   iter    |  target   | learni... | max_depth | n_esti... |
-------------------------------------------------------------
|  1        |  0.9754   |  0.3746   |  4.803    |  379.4    |
|  2        |  0.98     |  0.5987   |  1.624    |  120.2    |
|  3        |  0.9788   |  0.05818  |  4.465    |  320.5    |
|  4        |  0.9794   |  0.7081   |  1.082    |  486.5    |
|  5        |  0.978    |  0.045    |  1.427    |  50.11    |
|  6        |  0.9785   |  0.2694   |  1.39     |  50.08    |
|  7        |  0.9709   |  0.697    |  3.884    |  50.08    |
|  8        |  0.9799   |  0.3956   |  1.008    |  247.2    |
|  9        |  0.9799   |  0.6653   |  1.004    |  343.3    |
{'target': 0.979959536754115, 'params': {'learning_rate': 0.5986986183486169, 'max_depth': 1.624074561769746, 'n_estimators': 120.1975341512912}}


Fit a gradient boosting classifier with 100 decision stumps as weak learners

In [20]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.5, 
                                 max_depth=1, random_state=0).fit(train_features_data, train_targets_data)

Compare the classified data to the test set - returns percentage match

In [21]:
clf.score(test_features_data, test_targets_data)

0.9787709497206704